quick checks of the new fgdc triples structure
and maybe the triples (yes the triples, whatevs)

In [18]:
%reload_ext autoreload
%autoreload 2

import os
import json
import glob
from semproc.parser import Parser
from semproc.preprocessors.metadata_preprocessors import FgdcItemReader

In [43]:
# load the proto-triples example
with open('../response_examples/fgdc_proto_example_1.xml', 'r') as f:
    response = f.read()

# this shouldn't be necessary but cargo-culting here is fine by me.
response = response.replace('\\\n', '').replace('\r\n', '').replace('\\r', '').replace('\\n', '').replace('\n', '')
response = response.decode('utf-8', errors='replace').encode('unicode_escape') 
    
url = 'https://bluehub.jrc.ec.europa.eu/erddap/metadata/fgdc/xml/noaa_pfeg_d543_8870_bc7f_fgdc.xml'
identities = [
    {"protocol": "FGDC", 
     "metadata": {
            "version": ["FGDC Content Standards for Digital Geospatial Metadata, FGDC-STD-001-1998"], 
            "name": "FGDC"}
    }
]

parser = Parser(response)

In [44]:
# execute the parse (this one takes xml)
reader = FgdcItemReader(parser.xml)
description = reader.parse_item()

# for the faking of things
description['catalog_record']['url'] = url
description['catalog_record']['harvestDate'] = '2015-06-20T20:22:00.643Z'

In [45]:
description

{'catalog_record': {'conformsTo': 'http://www.ngdc.noaa.gov/metadata/published/xsd/ngdcSchema/schema.xsd',
  'harvestDate': '2015-06-20T20:22:00.643Z',
  'url': 'https://bluehub.jrc.ec.europa.eu/erddap/metadata/fgdc/xml/noaa_pfeg_d543_8870_bc7f_fgdc.xml'},
 'dataset': {'abstract': 'Navy Global Environmental Model (NAVGEM) is a global numerical weather prediction computer model. It replaced NOGAPS as the prime model in the middle of February 2013 at the Navy Fleet Numerical Meteorology and Oceanography Center (FNMOC) Weather model synoptic site. [Wikipedia]',
  'identifier': '',
  'spatial_extent': {'east': '180.0',
   'north': '90.0',
   'south': '-90.0',
   'west': '-180.0',
   'wkt': 'POLYGON ((-180 -90,-180 90,180 90,180 -90,-180 -90))'},
  'title': 'Navy Global Environmental Model (NAVGEM), 0.5 degree, Pressure MSL'},
 'keywords': [{'terms': ['conversion',
    'data',
    'fnmoc',
    'level',
    'mean',
    'navg',
    'observed',
    'pressure',
    'sea',
    'theortically',
  

In [57]:
# a minorly faked description dict (for the triples)

description = {
    "catalog_record": {
        "conformsTo": "http://www.ngdc.noaa.gov/metadata/published/xsd/ngdcSchema/schema.xsd",
        "harvestDate": "2015-06-20T20:22:00.643Z",
        "url": "https://bluehub.jrc.ec.europa.eu/erddap/metadata/fgdc/xml/noaa_pfeg_d543_8870_bc7f_fgdc.xml"
    },
    "dataset": {
        "abstract": "Navy Global Environmental Model (NAVGEM) is a global numerical weather prediction computer model. It replaced NOGAPS as the prime model in the middle of February 2013 at the Navy Fleet Numerical Meteorology and Oceanography Center (FNMOC) Weather model synoptic site. [Wikipedia]",
        "identifier": "",
        "spatial_extent": {
            "east": "180.0",
            "north": "90.0",
            "south": "-90.0",
            "west": "-180.0",
            "wkt": "POLYGON ((-180 -90,-180 90,180 90,180 -90,-180 -90))"
        },
        "title": "Navy Global Environmental Model (NAVGEM), 0.5 degree, Pressure MSL",
        "keywords": [
            {
                "terms": [
                    "conversion",
                    "data",
                    "fnmoc",
                    "level",
                    "mean",
                    "navg",
                    "observed",
                    "pressure",
                    "sea",
                    "theortically",
                    "value"
                ],
                "thesaurus": "Uncontrolled",
                "type": "theme"
            },
            {
                "terms": [
                    "time",
                    "latitude",
                    "longitude"
                ],
                "thesaurus": "CF Standard Name Table v27",
                "type": "theme"
            }
        ],
        "publisher": {
            "location": "Ispra, VA, Italy",
            "name": "ERDDAP, version 1.60, at Maritime Affairs Unit of the Joint Research Centre"
        },
        "temporal_extent": {
            "endDate": "20140813",
            "startDate": "20130215"
        },
        "webpages": [
            {"url": "https://bluehub.jrc.ec.europa.eu/erddap/griddap/noaa_pfeg_d543_8870_bc7f.html"},
            {"url": "https://bluehub.jrc.ec.europa.eu/erddap/griddap/noaa_pfeg_d543_8870_bc7f.graph"},
            {"url": "https://bluehub.jrc.ec.europa.eu/erddap/wms/noaa_pfeg_d543_8870_bc7f/request"}
        ]
    }
}

In [68]:
# let's play with triples

import rdflib
import hashlib
from uuid import uuid4
from rdflib import Graph, Literal, RDF, RDFS, Namespace, URIRef
from rdflib.namespace import DC, DCTERMS, FOAF, XSD, OWL


class Grapher():
    def __init__(self):
        self.graph = Graph()
        self._bind_namespaces()
    
    # some faked namespaces
    _ontology_uris = {
        'bcube': 'http://purl.org/nsidc/bcube/bcube#',
        'vcard': 'http://purl.org/nsidc/bcube/vcard#',
        'esip': 'http://purl.org/nsidc/bcube/esip#',
        'vivo': 'http://purl.org/nsidc/bcube/vivo#',
        'bibo': 'http://purl.org/nsidc/bcube/bibo#',
        'GeoSPARQL': 'http://purl.org/nsidc/bcube/geosparql#',
        'dcat': 'http://purl.org/nsidc/bcube/dcat#',
        'dc': str(DC),
        'dct': str(DCTERMS),
        'foaf': str(FOAF),
        'xsd': str(XSD),
        'owl': str(OWL)
    }
    
    def _bind_namespaces(self):
        # bind our lovely fake namespaces
        for prefix, uri in self._ontology_uris.iteritems():
            self.graph.bind(prefix, uri)

    def generate_predicate(self, prefix, name):
        return Namespace(self._ontology_uris[prefix])[name]
            
    def create_resource(self, resource_prefix, resource_type, identifier=''):
        # make a thing with a uuid as a urn
        # and just assign it to type if it's not overridden
        identifier = identifier if identifier else uuid4().urn
        resource = self.graph.resource(identifier)
        ref = Namespace(self._ontology_uris[resource_prefix])[resource_type]
        resource.add(OWL.a, URIRef(ref))
        return resource

    def serialize(self):
        return self.graph.serialize(format='turtle')
    
    def graphalize(self, doc):
        # not a word
        # so from our json.
        # which is currently incorrect in the relate
        # between the catalog record and the dataset (no primaryTopic ref)

        for root_entity_type, root_entity in doc.iteritems():
            if root_entity_type == 'catalog_record':
                catalog_record = self.create_resource('dcat', 'CatalogRecord', 'urn:sha:21a2c32fd74798563eed88a3fcd1acf38092f47e')
                catalog_record.add(self.generate_predicate('vcard', 'hasURL'), Literal(root_entity['url']))
                catalog_record.add(self.generate_predicate('vivo', 'harvestDate'), Literal(root_entity['harvestDate']))
                catalog_record.add(DC.conformsTo, Literal(root_entity['conformsTo']))
            elif root_entity_type == 'dataset':
                dataset = self.create_resource('dcat', 'Dataset', 'urn:sha:e018c0a1fb2f9fdc497f7f78eede802c833883ef')

                dataset.add(DCTERMS.identifier, Literal(root_entity['identifier']))
                dataset.add(DCTERMS.title, Literal(root_entity['title']))
                dataset.add(DC.description, Literal(root_entity['abstract']))

                if 'publisher' in root_entity:
                    publisher = self.create_resource('dcat', 'publisher')
                    publisher.add(DC.location, Literal(root_entity['publisher']['location']))
                    publisher.add(FOAF.name, Literal(root_entity['publisher']['name']))
                    dataset.add(self.generate_predicate('dcat', 'publisher'), publisher)

                for keywords in root_entity.get('keywords', []):
                    keyset = self.create_resource('bcube', 'thesaurusSubset')
                    if 'type' in keywords:
                        keyset.add(DC.hasType, Literal(keywords['type']))
                    if 'thesaurus' in keywords:
                        keyset.add(DC.partOf, Literal(keywords['thesaurus']))

                    for term in keywords['terms']:
                        keyset.add(self.generate_predicate('bcube', 'hasValue'), Literal(term))

                    dataset.add(DC.conformsTo, keyset)

                if 'temporal_extent' in root_entity:
                    # NOTE: these are not ISO 8601 obv.
                    begdate = root_entity['temporal_extent'].get('startDate')
                    enddate = root_entity['temporal_extent'].get('edDate')

                    dataset.add(self.generate_predicate('esip', 'startDate'), Literal(begdate))
                    dataset.add(self.generate_predicate('esip', 'endDate'), Literal(enddate))

                if 'spatial_extent' in root_entity:
                    dataset.add(DC.spatial, Literal(root_entity['spatial_extent']['wkt']))

                    dataset.add(self.generate_predicate('esip', 'westBound'),
                                Literal(root_entity['spatial_extent']['west']))

                    dataset.add(self.generate_predicate('esip', 'eastBound'),
                                Literal(root_entity['spatial_extent']['east']))

                    dataset.add(self.generate_predicate('esip', 'southBound'),
                                Literal(root_entity['spatial_extent']['south']))

                    dataset.add(self.generate_predicate('esip', 'northBound'),
                                Literal(root_entity['spatial_extent']['north']))

                for webpage in root_entity.get('webpages', []):
                    relation = self.create_resource('bibo', 'WebPage')
                    relation.add(self.generate_predicate('vcard', 'hasURL'), Literal(webpage['url']))
                    dataset.add(DC.relation, relation)
            else:
                continue

grapher = Grapher()    
grapher.graphalize(description)
print grapher.serialize()  


@prefix GeoSPARQL: <http://purl.org/nsidc/bcube/geosparql#> .
@prefix bcube: <http://purl.org/nsidc/bcube/bcube#> .
@prefix bibo: <http://purl.org/nsidc/bcube/bibo#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dcat: <http://purl.org/nsidc/bcube/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix esip: <http://purl.org/nsidc/bcube/esip#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix vcard: <http://purl.org/nsidc/bcube/vcard#> .
@prefix vivo: <http://purl.org/nsidc/bcube/vivo#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:sha:21a2c32fd74798563eed88a3fcd1acf38092f47e> dc:conformsTo "http://www.ngdc.noaa.gov/metadata/published/xsd/ngdcSchema/schema.xsd" ;
    vcard:hasURL "https://bluehub.jrc.ec.europa.eu/erddap/metadata/fgdc/xml

#### generating local graphs for any fgdc response

i would note that we are not catching every fgdc through the identifier - not quite valid/complete responses are being dropped. this generates roughly three thousand graphs.

In [ ]:
# so let's make a new temp class for the harvest + parser parts
# and this is not what we really want to do. probably.

class Fgdc():
    def __init__(self, doc):
        self.url = doc.get('source_url', '')
        self.response = self._prep_response(doc.get('content', ''))
        self.harvested = doc.get('tstamp', '')  # except i did not carry this through in the clean task
        self._prep_parser()
        
    def _prep_parser(self):
        parser = Parser(self.response)
        self.reader = FgdcItemReader(parser.xml)
    
    def _prep_response(self, response):
        # this shouldn't be necessary but cargo-culting here is fine by me.
        response = response.replace('\\\n', '').replace('\r\n', '').replace('\\r', '').replace('\\n', '').replace('\n', '')
        return response.decode('utf-8', errors='replace').encode('unicode_escape') 
        
    def parse(self):
        description = self.reader.parse_item()
        
        # note: this is still very very wrong
        description['catalog_record']['url'] = self.url
        description['catalog_record']['harvestDate'] = self.harvested
        
        return description

In [ ]:
import csv

identified_dir = '/Users/sscott/Documents/tmp/identified/'
triples_dir = '/Users/sscott/Documents/tmp/triples/'

with open('data/small_harvest_fgdc_as_md5.csv', 'rb') as csvfile:
    cr = csv.DictReader(csvfile, delimiter="|")

    for row in cr:
        url = row['source_url'].strip()
        md5 = row['raw_content_md5'].strip()
        
        # go open the cleaned up version by md5
        filepath = os.path.join(identified_dir, md5 + '_identified.json')
        if not os.path.exists(filepath):
            continue
        
        with open(filepath, 'r') as f:
            data = json.loads(f.read())
            
        fgdc = Fgdc(data)
        description = fgdc.parse()
        
        grapher = Grapher()
        grapher.graphalize(description)
        ttl = grapher.serialize()
        
        with open(os.path.join(triples_dir, md5 + '.ttl'), 'w') as f:
            f.write(ttl)